<a href="https://colab.research.google.com/github/Zen-Teng/Stylos/blob/main/FineTuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Unsloth

In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-uxipv91l/unsloth_6cad6d77323f498ca4cd548710dd3cd8
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-uxipv91l/unsloth_6cad6d77323f498ca4cd548710dd3cd8
  Resolved https://github.com/unslothai/unsloth.git to commit 07dbd8620b096dae11dfbaacb2120493975d24d4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Install beautifulsoup4

In [2]:
!pip install ebooklib beautifulsoup4

In [3]:
#### Load Existing model using Unsloth and give one example from Mergers and Acquistions News

from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(model_name = "unsloth/llama-3-8b-bnb-4bit", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference


mna_news_input = """HCL Technologies has completed acquisition of 51% stake in German IT Consulting Company, Gesellschaft für Banksysteme GmbH (“GBS”). The acquisition was completed on January 05, 2022."""
mna_news_instruction = """Which company is the Acquirer."""


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.2.1: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Hello World! Just to test if llama-3 is working

In [4]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [5]:
inputs = tokenizer([alpaca_prompt.format(mna_news_instruction, mna_news_input, "")], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 90)
tokenizer.batch_decode(outputs)[0]

'<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhich company is the Acquirer.\n\n### Input:\nHCL Technologies has completed acquisition of 51% stake in German IT Consulting Company, Gesellschaft für Banksysteme GmbH (“GBS”). The acquisition was completed on January 05, 2022.\n\n### Response:\nHCL Technologies has completed acquisition of 51% stake in German IT Consulting Company, Gesellschaft für Banksysteme GmbH (“GBS”). The acquisition was completed on January 05, 2022.<|end_of_text|>'

Data Taken from Anna's Archive
1. Convert EPUB to TXT using beautifulsoup4

In [6]:
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup
import re

def epub_to_clean_text(epub_path, output_path):
    book = epub.read_epub(epub_path)
    full_text = []

    # Iterate through every item in the book
    for item in book.get_items():
        # We only care about proper document chapters (ignore images/css)
        if item.get_type() == ebooklib.ITEM_DOCUMENT:

            # 1. Parse HTML
            soup = BeautifulSoup(item.get_body_content(), 'html.parser')

            # 2. Extract text (strip tags)
            text = soup.get_text(separator='\n')

            # 3. CLEANING HEURISTICS (The Secret Sauce)

            # Skip "Junk" Chapters based on content length or keywords
            # Most actual chapters are longer than 500 characters
            if len(text) < 500:
                continue

            # Skip Copyright/Table of Contents/Dedication pages
            if any(x in text.lower() for x in ['copyright', 'all rights reserved', 'table of contents']):
                continue

            # 4. Normalize Whitespace
            # Turn multiple newlines into a single newline to maintain flow
            text = re.sub(r'\n+', '\n', text).strip()

            full_text.append(text)

    # Save to file
    with open(output_path, "w", encoding="utf-8") as f:
        # Join chapters with a distinct separator (optional, but helps context)
        f.write("\n\n".join(full_text))

    print(f"Success! Processed text saved to {output_path}")

# --- USAGE ---
# Replace with your actual filename
epub_to_clean_text("Harry_Potter_and_the_Half-Blood_Prince.epub", "training_data.txt")

Success! Processed text saved to training_data.txt


Training the model with unsloth

In [9]:
import torch
import gc

# 1. Delete the specific variables if they exist in memory
# (Add 'trainer' to this list if you have already initialized it)
try:
    del model
    del tokenizer
except NameError:
    pass

# 2. Run Python's garbage collection to clean up the deleted objects
gc.collect()

# 3. Clear the cache from the GPU
torch.cuda.empty_cache()

In [13]:
from unsloth import FastLanguageModel
from datasets import load_dataset

# 1. Load your processed text file
dataset = load_dataset("text", data_files={"train": "training_data.txt"}, split="train")

# 2. Tokenize it (Unsloth handles the splitting here)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", # or Mistral
    max_seq_length = 2048,
    load_in_4bit = True,
)
# 2.a Attach the LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

def formatting_func(examples):
    # This chunks the text into blocks of 2048 tokens
    return {"text": examples["text"]}

# 3. Set up the Trainer (Group texts together)
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    tokenizer = tokenizer,
    dtype = None,           # Auto-detection
    load_in_4bit = True,    # Ensure 4bit is strictly True
    # force the model to use the GPU
    device_map = "cuda",
    packing = True, # <--- IMPORTANT: This combines short chapters into full blocks
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        max_steps = 60, # Adjust based on how much data you have
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        output_dir = "outputs",
        optim = "adamw_8bit",
    ),
)

trainer.train()

==((====))==  Unsloth 2026.2.1: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 9,967 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
1,3.410800
2,3.927800
3,2.945300
4,3.100800
5,3.571300
6,3.636700
7,2.590800
8,3.129600
9,2.950200
10,3.067600


wandb: WARNING URL not available in offline run


train/epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▇▇▇▇▇▁▁▂▂▂▂▂▃▃▃▄▅▅▅▅▆▆▆▇█
train/global_step,▁▁▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▆▆▆██▁▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇▇█
train/grad_norm,▂▁▁▃▃▂▄▅▇▂▃▃▄▄▅▂▃▃▃▃▃▃▄▃▃▃▃▇▅▄▅▃▄▃▃▃▃█▃▃
train/learning_rate,█▇▇▇▆▆▆▅▅▅▅▄▄▄▃▃▃▂▂▂███▇▇▆▆▅▅▅▄▄▄▄▄▃▃▂▂▁
train/loss,▅▃▆▄█▆▃▂▅▄▄▃▄▃▃▁▆▄▃▄▆█▃▄▂▃▄▄▁▄▅▃▂▄▃▃▃▂▄▃
total_flos,538257698955264.0
train/epoch,0.04816
train/global_step,60
train/grad_norm,2.57867
train/learning_rate,1e-05
train/loss,2.6191


TrainOutput(global_step=60, training_loss=2.8975619355837505, metrics={'train_runtime': 207.0717, 'train_samples_per_second': 2.318, 'train_steps_per_second': 0.29, 'total_flos': 538257698955264.0, 'train_loss': 2.8975619355837505, 'epoch': 0.04815892445068727})

In [ ]:
import json

# 1. Load the raw data (Mocking Role 1's output)
with open('Mock_Data.json', 'r') as f:
    raw_data = json.load(f)

# 2. Define the "System Prompt" (The instruction to the model)
# This teaches the model: "When I ask you to write in my style, produce this text."
system_instruction = "You are a personal ghostwriter. Rewrite the following text in the user's specific voice and style."

# 3. Open the output file (JSONL format - JSON Lines)
with open('training.jsonl', 'w') as outfile:
    for entry in raw_data:
        # We create a training pair.
        # "instruction": What the user asks
        # "input": Context, here we can leave it empty or put a generic tag
        # "output": The actual text from your file

        training_sample = {
            "instruction": system_instruction,
            "input": "",
            "output": entry['text']
        }

        # Write to file (one JSON object per line)
        json.dump(training_sample, outfile)
        outfile.write('\n')

print("✅ Success! Converted data.json -> training.jsonl")
print("Preview of first line:")
with open('training.jsonl', 'r') as f:
    print(f.readline())